In [1]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

4L_submission[0.06699597124671304, 0.97892590074779062].csv
4L_submission.csv
sub201801042008.csv
sub201801042044.csv
test_submission_0.15153_180104.csv



In [2]:
print(check_output(["ls"]).decode("utf8"))

6075707.zip
6126783.zip
6300793.zip
6304456.zip
6338977.zip
input
Untitled.ipynb



In [3]:
sub_path = "./input"
all_files = os.listdir(sub_path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4
0,5941774d,0.023214,0.002585,1.758107e-01,0.003044,0.008796
1,4023181e,0.459038,0.996710,4.762523e-01,0.990582,0.964124
2,b20200e4,0.005891,0.002125,6.421022e-07,0.006188,0.072836
3,e7f018bb,0.997501,0.999916,9.970161e-01,0.999728,0.995568
4,4371c8c3,0.016640,0.994338,2.452419e-02,0.998925,0.042753


In [4]:
# check correlation
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4
is_iceberg_0,1.000000,0.656438,0.922837,0.671522,0.754505
is_iceberg_1,0.656438,1.000000,0.581046,0.926807,0.703777
is_iceberg_2,0.922837,0.581046,1.000000,0.610635,0.651560
is_iceberg_3,0.671522,0.926807,0.610635,1.000000,0.707642
is_iceberg_4,0.754505,0.703777,0.651560,0.707642,1.000000


In [5]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

In [6]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

# Mean Stacking LB0.1492

In [7]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_mean']
concat_sub[['id', 'is_iceberg']].to_csv('stack_mean.csv', 
                                        index=False, float_format='%.6f')

# Median Stacking LB0.1473

In [8]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
concat_sub[['id', 'is_iceberg']].to_csv('stack_median.csv', 
                                        index=False, float_format='%.6f')

# PushOut + Median Stacking LB0.2773

In [9]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 1, 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             0, concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_pushout_median.csv', 
                                        index=False, float_format='%.6f')

# MinMax + Mean Stacking LB0.1437

In [15]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_mean']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_mean.csv', 
                                        index=False, float_format='%.6f')

# MinMax + Median Stacking LB0.1451

In [16]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_median.csv', 
                                        index=False, float_format='%.6f')

In [17]:
concat_sub

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,is_iceberg,is_iceberg_base
0,5941774d,2.321353e-02,2.585285e-03,1.758107e-01,3.044423e-03,0.008796,0.175811,2.585285e-03,0.042690,8.795977e-03,2.585285e-03,1.758107e-01
1,4023181e,4.590382e-01,9.967103e-01,4.762523e-01,9.905820e-01,0.964124,0.996710,4.590382e-01,0.777341,9.641239e-01,9.641239e-01,4.762523e-01
2,b20200e4,5.891184e-03,2.125204e-03,6.421022e-07,6.188220e-03,0.072836,0.072836,6.421022e-07,0.017408,5.891184e-03,6.421022e-07,6.421022e-07
3,e7f018bb,9.975005e-01,9.999158e-01,9.970161e-01,9.997279e-01,0.995568,0.999916,9.955682e-01,0.997946,9.975005e-01,9.999158e-01,9.970161e-01
4,4371c8c3,1.663989e-02,9.943380e-01,2.452419e-02,9.989253e-01,0.042753,0.998925,1.663989e-02,0.415436,4.275284e-02,4.275284e-02,2.452419e-02
5,a8d9b1fd,3.366164e-01,8.705235e-01,3.649666e-01,6.959881e-01,0.699916,0.870523,3.366164e-01,0.593602,6.959881e-01,6.959881e-01,3.649666e-01
6,29e7727e,2.937307e-02,1.443777e-02,1.054916e-02,2.132297e-02,0.185793,0.185793,1.054916e-02,0.052295,2.132297e-02,1.054916e-02,1.054916e-02
7,92a51ffb,9.970354e-01,9.998017e-01,9.999193e-01,9.999089e-01,0.998004,0.999919,9.970354e-01,0.998934,9.998017e-01,9.999193e-01,9.999193e-01
8,c769ac97,3.507295e-06,5.343305e-06,6.406796e-07,4.096438e-07,0.010134,0.010134,4.096438e-07,0.002029,3.507295e-06,4.096438e-07,6.406796e-07
9,aee0547d,4.131402e-05,2.257358e-05,4.235656e-05,1.208634e-05,0.001960,0.001960,1.208634e-05,0.000416,4.131402e-05,1.208634e-05,4.235656e-05


# MinMax + BestBase Stacking LB0.1538

In [12]:
# load the model with best base performance
sub_base = pd.read_csv('./input/sub201801042044.csv')

In [13]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_bestbase.csv', 
                                        index=False, float_format='%.6f')

In [14]:
concat_sub

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_max,is_iceberg_min,is_iceberg_mean,is_iceberg_median,is_iceberg,is_iceberg_base
0,5941774d,2.321353e-02,2.585285e-03,1.758107e-01,3.044423e-03,0.008796,0.175811,2.585285e-03,0.042690,8.795977e-03,2.585285e-03,1.758107e-01
1,4023181e,4.590382e-01,9.967103e-01,4.762523e-01,9.905820e-01,0.964124,0.996710,4.590382e-01,0.777341,9.641239e-01,4.762523e-01,4.762523e-01
2,b20200e4,5.891184e-03,2.125204e-03,6.421022e-07,6.188220e-03,0.072836,0.072836,6.421022e-07,0.017408,5.891184e-03,6.421022e-07,6.421022e-07
3,e7f018bb,9.975005e-01,9.999158e-01,9.970161e-01,9.997279e-01,0.995568,0.999916,9.955682e-01,0.997946,9.975005e-01,9.999158e-01,9.970161e-01
4,4371c8c3,1.663989e-02,9.943380e-01,2.452419e-02,9.989253e-01,0.042753,0.998925,1.663989e-02,0.415436,4.275284e-02,2.452419e-02,2.452419e-02
5,a8d9b1fd,3.366164e-01,8.705235e-01,3.649666e-01,6.959881e-01,0.699916,0.870523,3.366164e-01,0.593602,6.959881e-01,3.649666e-01,3.649666e-01
6,29e7727e,2.937307e-02,1.443777e-02,1.054916e-02,2.132297e-02,0.185793,0.185793,1.054916e-02,0.052295,2.132297e-02,1.054916e-02,1.054916e-02
7,92a51ffb,9.970354e-01,9.998017e-01,9.999193e-01,9.999089e-01,0.998004,0.999919,9.970354e-01,0.998934,9.998017e-01,9.999193e-01,9.999193e-01
8,c769ac97,3.507295e-06,5.343305e-06,6.406796e-07,4.096438e-07,0.010134,0.010134,4.096438e-07,0.002029,3.507295e-06,4.096438e-07,6.406796e-07
9,aee0547d,4.131402e-05,2.257358e-05,4.235656e-05,1.208634e-05,0.001960,0.001960,1.208634e-05,0.000416,4.131402e-05,1.208634e-05,4.235656e-05
